### Proccessing of UNData on Energy Statistics

In [ ]:
import numpy as np
import pandas as pd
import iode as io

wk_path = "to define"

# Load and merge UNData into one dataframe (due to downlad limtiations)
pathwork = wk_path + "Data_Raw/Energy/UNData/"

# Create lists with files' name
lst_endfiles = ["09_24_43","09_28_09","09_30_55","09_32_41","09_50_20","09_53_15","09_55_38","10_05_44"]
lst_files = [f"UNSD+DF_UNDATA_ENERGY+1.2_2025_03_13_{endfiles}.csv" for endfiles in lst_endfiles]

# Load each file in a datafarme and store them in to "full_eb" 
full_eb =pd.DataFrame()
for files in lst_files:
    pathdata = pathwork+files
    data_tmp = pd.read_csv(pathdata, sep=";", header = 0, dtype= str)
    data_tmp[["OBS_VALUE", "UNIT_MULT", "CONVERSION_FACTOR"]] = data_tmp[["OBS_VALUE", "UNIT_MULT", "CONVERSION_FACTOR"]].astype('float')
    data_tmp[["TIME_PERIOD"]] = data_tmp[["TIME_PERIOD"]].astype('int')
    full_eb = pd.concat([full_eb, data_tmp])

# Clean dataframe
full_eb.drop(columns= ["DATAFLOW", "FREQ", "OBS_STATUS"], inplace = True)
full_eb
full_eb = full_eb.rename(columns = {"REF_AREA": "Region_un_cd", "COMMODITY": "Commodity_un_cd", "TRANSACTION": "Variable_un_cd", 
                                    "TIME_PERIOD": "Time", "OBS_VALUE": "Value", "UNIT_MULT": "Unit_mult", "UNIT_MEASURE": "Unit", 
                                    "CONVERSION_FACTOR": "Conversion_factor"})



In [ ]:
## Browse dataframe content
print(full_eb.columns)
for content in full_eb.columns:
    print(content, ": ", full_eb[content].unique())


In [ ]:
# To map regions/commodities/variables' codes with names stored in csv files (Regions_list, Commodities_list, Variables_list)
dict_map_nm = {"reg_lst": ["Regions_list.csv", "Region_un_cd"], "cmd_lst": ["Commodities_list.csv", "Commodity_un_cd"], "var_lst": ["Variables_list.csv", "Variable_un_cd"]}

eb_trav = full_eb.copy()

for df_nm, files in dict_map_nm.items():
    file_nm = files[0]
    var_ref = files[1] 
    pathdata = pathwork+file_nm 
    df_tmp = pd.read_csv(pathdata, sep=";", header = 0, dtype=str)
    eb_trav = eb_trav.merge(df_tmp, on=var_ref , how= "left")


eb_trav = eb_trav[["Region_un_cd", "Region_un_nm", "Region_NeW_cd", "Commodity_un_cd", "Commodity_un_nm", "Commodity_NeW_nm", "Variable_un_cd", "Variable_un_nm", "Variable_NeW", "Time", "Value", "Unit_mult", "Unit", "Conversion_factor"]]

# Convert all value in TJ (using conversion factor) / Caution for capacity variables
eb_trav["Value_TJ"] = eb_trav["Value"]*eb_trav["Conversion_factor"]


In [ ]:
# Creation of DFINENP & DINTENP by energy product and energy sector
lst_enr_var = ["DFINENP", "DINTENP"]

df_enr_NeW = pd.DataFrame()
for var in lst_enr_var: 
    # Select rows for which column "Variable_New" values start with "DFINENP" or "DINTENP", "Variable_NeW" have been defined manually in mapping "Variables_list.csv" file
    eb_tmp = eb_trav.loc[(eb_trav["Variable_NeW"].str.startswith(f"{var}", na=False))].copy()
    # Grouping to by NeW Energy commodities as defined in mapping "Commodities_list.csv" file
    eb_tmp = eb_tmp.groupby(by=['Region_un_cd', 'Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_cd',
                                    'Variable_un_nm', 'Variable_NeW'], as_index=False)["Value_TJ"].sum()
    # Extract string following DFINENP_ or DINTENP_ to create two different columns: one for the variable name (columns; "Variable_NeW_cd") and the second for the sector (columns; "Sector_agg")
    suf_var_nem = eb_tmp['Variable_NeW'].str.extract(fr'{var}_(.*)', expand=False)
    eb_tmp["Variable_NeW_cd"] = f"{var}"
    eb_tmp["Sector_agg"] = suf_var_nem 
    eb_tmp.drop(columns=["Region_un_cd", "Variable_un_cd", "Variable_NeW"], inplace = True)
    # Add the dataframe to the general one "df_enr_NeW"
    df_enr_NeW = pd.concat([df_enr_NeW, eb_tmp])

df_enr_NeW[["Region_un_nm", "Region_NeW_cd", "Variable_un_nm", "Variable_NeW_cd", "Sector_agg", "Commodity_NeW_nm",	"Value_TJ"]]    


In [ ]:
# Create PRODPELEC & PRODPHEAT (Only for ELEC and HEAT commodity)
# List of code for variables "Production from ..."
lst_prodp_therm = ["01BI","01CL","01CP","01CR","01DL","01LB","01LBF","01MG","01NG","01NRW","01OS","01PP","01PT","01RF","01RW","01SBF","01BS"]
lst_prodp_ntherm = ["015EB", "015G", "015HP", "015HY", "015N", "015O", "015S", "015SP", "015ST", "015T", "015W",
                     "016G", "016HY", "016N", "016O", "016S", "016SP", "016ST", "016T", "016W"]

lst_prodp_pgheat = lst_prodp_therm + lst_prodp_ntherm 

eb_tmp = pd.DataFrame()
eb_tmp = eb_trav.loc[eb_trav["Variable_un_cd"].isin(lst_prodp_pgheat)].copy()
# Grouping to by NeW energy commodities as defined in mapping "Commodities_list.csv" file
eb_tmp = eb_tmp.groupby(by=['Region_un_cd', 'Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_cd',
                                    'Variable_un_nm', 'Variable_NeW'], as_index=False)["Value_TJ"].sum()


# Update column "Variable_NeW_cd" by concatenate PRODP with ELEC or HEAT and create column "Sector_agg" accordingly
eb_tmp["Variable_NeW_cd"] = "PRODP"+ eb_tmp["Commodity_NeW_nm"]
eb_tmp.loc[(eb_tmp["Commodity_NeW_nm"] == "ELEC"), "Sector_agg"] = "PG" 
eb_tmp.loc[(eb_tmp["Commodity_NeW_nm"] == "HEAT"), "Sector_agg"] = "HT"


# Extract string following PRODP_ to update columns "Variable_un_nm" and "Commodity_NeW_nm"
suf_var_nem = eb_tmp['Variable_NeW'].str.extract(r'PRODP(.*)', expand=False)
eb_tmp["Variable_un_nm"] = eb_tmp["Commodity_NeW_nm"] + " production from " + suf_var_nem.str[1:]
eb_tmp["Commodity_NeW_nm"] = suf_var_nem.str[1:]

# Grouping to by keep only one entry by region, variable, sector and commodity
eb_tmp = eb_tmp.groupby(by=["Region_un_nm", "Region_NeW_cd", "Variable_un_nm", "Variable_NeW_cd",
                             "Sector_agg", "Commodity_NeW_nm"], as_index=False)["Value_TJ"].sum()

# Add temporary dataframe into global one 
# ⚠️ Do not to run several times
df_enr_NeW = pd.concat([df_enr_NeW, eb_tmp])



In [ ]:
# PRODP except HEAT & ELEC
lst_elecht = {"ELEC", "HEAT"}

eb_tmp = pd.DataFrame()
# Select all variables start with "Production from ..." but exclude ELEC and HEAT commodities
eb_tmp = eb_trav.loc[(eb_trav["Variable_un_cd"] == "01") & (~eb_trav["Commodity_NeW_nm"].isin(lst_elecht))].copy()
# Grouping to by NeW Energy commodities as defined in mapping "Commodities_list.csv" file
eb_tmp = eb_tmp.groupby(by=['Region_un_cd', 'Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_cd',
                                    'Variable_un_nm', 'Variable_NeW'], as_index=False)["Value_TJ"].sum()

# Update column "Variable_NeW_cd" as column "Variable_NeW" and "Sector_agg" as "Commodity_NeW_nm"
eb_tmp["Variable_NeW_cd"] = eb_tmp["Variable_NeW"]
eb_tmp["Sector_agg"] = eb_tmp["Commodity_NeW_nm"]
eb_tmp = eb_tmp.groupby(by=["Region_un_nm", "Region_NeW_cd","Variable_NeW_cd","Sector_agg","Commodity_NeW_nm"], as_index=False)["Value_TJ"].sum()
eb_tmp["Variable_un_nm"] = "Primary production of " + eb_tmp["Commodity_NeW_nm"]

# Add temporary dataframe into global one 
# ⚠️ Do not to run several times
df_enr_NeW = pd.concat([df_enr_NeW, eb_tmp])


In [ ]:
# Transform dataframe df_enr_NeW to split values (DFINENP & DINTENP) for sector PGHEAT between PG and HEAT
# In case of DFINENP (simple as information available in column "Commodity_NeW_nm")
df_enr_NeW.loc[(df_enr_NeW["Sector_agg"] == "PGHEAT") & (df_enr_NeW["Variable_NeW_cd"] == "DFINENP")
                                  & (df_enr_NeW["Commodity_NeW_nm"] == "ELEC"), "Sector_agg"] = "PG"
df_enr_NeW.loc[(df_enr_NeW["Sector_agg"] == "PGHEAT") & (df_enr_NeW["Variable_NeW_cd"] == "DFINENP")
                                  & (df_enr_NeW["Commodity_NeW_nm"] == "HEAT"), "Sector_agg"] = "HT"

# Thermal input used for PG & HEAT 
lst_input_therm = ['COMB', 'OIL', 'GAS', 'WD', 'MW', 'IW', 'SBM', 'LBF', 'BG', 'SOLT', 'GEO', 'HEAT', "WST"]

# Subset for DINTENP & DFINENP with column "Sector_agg" = "PGHEAT" & # Subset for DINTENP & DFINENP with column "Sector_agg" different from "PGHEAT" (to merge at the end)
df_dinten_pgheat = df_enr_NeW.loc[(df_enr_NeW["Commodity_NeW_nm"].isin(lst_input_therm)) & (df_enr_NeW["Sector_agg"] == "PGHEAT") & (df_enr_NeW["Variable_NeW_cd"] == "DINTENP")].copy()
df_non_dinten_pgheat = df_enr_NeW.loc[~((df_enr_NeW["Commodity_NeW_nm"].isin(lst_input_therm)) & (df_enr_NeW["Sector_agg"] == "PGHEAT") & (df_enr_NeW["Variable_NeW_cd"] == "DINTENP"))].copy()
df_dfinen_pgheat = df_non_dinten_pgheat.loc[(df_non_dinten_pgheat["Commodity_NeW_nm"].isin(lst_input_therm)) & (df_non_dinten_pgheat["Sector_agg"] == "PGHEAT") & (df_non_dinten_pgheat["Variable_NeW_cd"] == "DFINENP")].copy()
df_non_dfinen_pgheat = df_non_dinten_pgheat.loc[~((df_non_dinten_pgheat["Commodity_NeW_nm"].isin(lst_input_therm)) & (df_non_dinten_pgheat["Sector_agg"] == "PGHEAT") & (df_non_dinten_pgheat["Variable_NeW_cd"] == "DFINENP"))].copy()

# Subset of PRODPELEC and PRODPHEAT grouped by region
df_propagg = df_enr_NeW.loc[(df_enr_NeW["Commodity_NeW_nm"].isin(lst_input_therm)) 
                            & ((df_enr_NeW["Variable_NeW_cd"] == "PRODPELEC") | (df_enr_NeW["Variable_NeW_cd"] == "PRODPHEAT"))].copy()

df_propagg = df_propagg.groupby(by=["Region_un_nm", "Region_NeW_cd", "Variable_NeW_cd", "Commodity_NeW_nm"], as_index=False)["Value_TJ"].sum()

# Pivoted dataframe for PRODPELEC & PRODPHEAT values
df_propagg_piv = df_propagg.pivot_table(index=['Region_un_nm', 'Region_NeW_cd', "Commodity_NeW_nm"], columns='Variable_NeW_cd', values='Value_TJ').reset_index()
df_propagg_piv.columns.name = None  # Remove names of index
df_propagg_piv = df_propagg_piv.rename(columns={'PRODPELEC': 'Value_TJ_PRODPELEC', 'PRODPHEAT': 'Value_TJ_PRODPHEAT'})
df_propagg_piv.loc[df_propagg_piv["Value_TJ_PRODPHEAT"].isna(), "Value_TJ_PRODPHEAT"] = 0 
df_propagg_piv.loc[df_propagg_piv["Value_TJ_PRODPELEC"].isna(), "Value_TJ_PRODPELEC"] = 0

# Merge subset of DINTENP with subset of PRODPELEC & PRODPHEAT to calculated DINTENP for ELEC & HEAT
df_dinten_mrg = pd.merge(df_dinten_pgheat,
                          df_propagg_piv[['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Value_TJ_PRODPELEC', 'Value_TJ_PRODPHEAT']],
                          on=['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm'],
                          how='left')

df_dinten_mrg = df_dinten_mrg[['Region_un_nm', 'Region_NeW_cd', 'Variable_un_nm', 'Sector_agg', 'Commodity_NeW_nm', 'Value_TJ_PRODPELEC', 'Value_TJ_PRODPHEAT', 'Value_TJ']]
df_dinten_mrg["Value_DINTENP_PG_TJ"] = df_dinten_mrg["Value_TJ"]*(df_dinten_mrg["Value_TJ_PRODPELEC"]/(df_dinten_mrg["Value_TJ_PRODPELEC"]+df_dinten_mrg["Value_TJ_PRODPHEAT"]))
df_dinten_mrg["Value_DINTENP_HT_TJ"] = df_dinten_mrg["Value_TJ"]*(df_dinten_mrg["Value_TJ_PRODPHEAT"]/(df_dinten_mrg["Value_TJ_PRODPELEC"]+df_dinten_mrg["Value_TJ_PRODPHEAT"]))

##
# Special case of IW (as PRODPELEC/PRODPHEAT for not exist for IW in raw data)
for reg in df_dinten_mrg["Region_un_nm"].unique():
    # Filter for IW pby region
    mask_iw = (df_dinten_mrg["Region_un_nm"] == reg) & (df_dinten_mrg["Commodity_NeW_nm"] == "IW")
    if not df_dinten_mrg.loc[mask_iw].empty: # apply only when missing values
        val_iw_arr = df_dinten_mrg.loc[mask_iw, "Value_TJ"].values # Extract IW values
        if len(val_iw_arr) != 1:
            continue  
        val_iw = val_iw_arr[0]

        mask_mw = (df_dinten_mrg["Region_un_nm"] == reg) & (df_dinten_mrg["Commodity_NeW_nm"] == "MW") # Extract PRODPELEC + PRODPHEAT for MW
        prod_elec = df_dinten_mrg.loc[mask_mw, "Value_TJ_PRODPELEC"].values
        prod_heat = df_dinten_mrg.loc[mask_mw, "Value_TJ_PRODPHEAT"].values

        # In case no value, replace by 
        prod_elec_val = prod_elec[0] if len(prod_elec) else df_dinten_mrg.loc[(df_dinten_mrg["Region_un_nm"] == reg) , "Value_TJ_PRODPELEC"].sum()
        prod_heat_val = prod_heat[0] if len(prod_heat) else df_dinten_mrg.loc[(df_dinten_mrg["Region_un_nm"] == reg) , "Value_TJ_PRODPHEAT"].sum()
        prod_elec_val = 0 if pd.isna(prod_elec_val) else prod_elec_val
        prod_heat_val = 0 if pd.isna(prod_heat_val) else prod_heat_val

        denom = prod_elec_val + prod_heat_val
        if denom == 0:
            continue  # avoid div by 0

        # Change values
        df_dinten_mrg.loc[mask_iw, "Value_DINTENP_PG_TJ"] = val_iw * (prod_elec_val / denom)
        df_dinten_mrg.loc[mask_iw, "Value_DINTENP_HT_TJ"] = val_iw * (prod_heat_val / denom)


###
# Merge subset of DFINENP with subset of PRODPELEC & PRODPHEAT to calculated DFINENP for ELEC & HEAT
df_dfinen_mrg = pd.merge(df_dfinen_pgheat,
                          df_propagg_piv[['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Value_TJ_PRODPELEC', 'Value_TJ_PRODPHEAT']],
                          on=['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm'],
                          how='left')
df_dfinen_mrg = df_dfinen_mrg[['Region_un_nm', 'Region_NeW_cd', 'Variable_un_nm', 'Sector_agg', 'Commodity_NeW_nm', 'Value_TJ_PRODPELEC', 'Value_TJ_PRODPHEAT', 'Value_TJ']]
df_dfinen_mrg["Value_DFINENP_PG_TJ"] = df_dfinen_mrg["Value_TJ"]*(df_dfinen_mrg["Value_TJ_PRODPELEC"]/(df_dfinen_mrg["Value_TJ_PRODPELEC"]+df_dfinen_mrg["Value_TJ_PRODPHEAT"]))
df_dfinen_mrg["Value_DFINENP_HT_TJ"] = df_dfinen_mrg["Value_TJ"]*(df_dfinen_mrg["Value_TJ_PRODPHEAT"]/(df_dfinen_mrg["Value_TJ_PRODPELEC"]+df_dfinen_mrg["Value_TJ_PRODPHEAT"]))


df_dfinen_mrg.loc[(df_dfinen_mrg["Region_NeW_cd"] == "WE") & (df_dfinen_mrg["Commodity_NeW_nm"] == "SBM") 
                   & (df_dfinen_mrg["Region_un_nm"] == "Norway including Svalbard and Jan Mayen Islands")]

# Unpivoted dataframe for DINTENP for ELEC & HEAT
df_dinten_trf = pd.melt(df_dinten_mrg, id_vars=['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_nm'],
                        value_vars=['Value_DINTENP_PG_TJ', 'Value_DINTENP_HT_TJ'], var_name='Sector_agg', value_name='Value')


# Unpivoted dataframe for DFINENP for ELEC & HEAT
df_dfinen_trf = pd.melt(df_dfinen_mrg, id_vars=['Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_nm'],
                        value_vars=['Value_DFINENP_PG_TJ', 'Value_DFINENP_HT_TJ'], var_name='Sector_agg', value_name='Value')

# Remplace name of value in column "Sector_agg" by "PG" & "HT" accordingly
df_dinten_trf['Sector_agg'] = df_dinten_trf['Sector_agg'].replace({'Value_DINTENP_PG_TJ': 'PG', 'Value_DINTENP_HT_TJ': 'HT'})
df_dinten_trf.rename(columns= {'Value': "Value_TJ"}, inplace=True)
df_dfinen_trf['Sector_agg'] = df_dfinen_trf['Sector_agg'].replace({'Value_DFINENP_PG_TJ': 'PG', 'Value_DFINENP_HT_TJ': 'HT'})
df_dfinen_trf.rename(columns= {'Value': "Value_TJ"}, inplace=True)

# Add a column "Variable_NeW_cd with appropriate name to merge in df_enr_NeW"
df_dinten_trf["Variable_NeW_cd"] = "DINTENP"
df_dfinen_trf["Variable_NeW_cd"] = "DFINENP"

# Add temporary dataframe into global one 
# ⚠️ Do not to run several times
df_enr_NeW =  pd.concat([df_non_dfinen_pgheat, df_dfinen_trf, df_dinten_trf])


In [ ]:
# Other varianles DSTOCK, IMPP, EXPP, DNONENP, DNONENP
eb_trav["Variable_NeW"].unique()
lst_enr_var = ["IMPP", "EXPP", "DSTOCKP", "DNONENP", "LOSSESP"]

eb_tmp = pd.DataFrame()
eb_tmp = eb_trav.loc[(eb_trav["Variable_NeW"].isin(lst_enr_var))].copy()
eb_tmp = eb_tmp.groupby(by=['Region_un_cd', 'Region_un_nm', 'Region_NeW_cd', 'Commodity_NeW_nm', 'Variable_un_cd',
                                    'Variable_un_nm', 'Variable_NeW'], as_index=False)["Value_TJ"].sum()
eb_tmp["Variable_NeW_cd"] = eb_tmp["Variable_NeW"]
eb_tmp["Sector_agg"] = eb_tmp["Commodity_NeW_nm"]
eb_tmp = eb_tmp.groupby(by=["Region_un_nm", "Region_NeW_cd", "Variable_un_nm", "Variable_NeW_cd","Sector_agg","Commodity_NeW_nm"], as_index=False)["Value_TJ"].sum()
df_enr_NeW = pd.concat([df_enr_NeW, eb_tmp])

df_enr_NeW[["Region_un_nm", "Region_NeW_cd", "Variable_un_nm", "Variable_NeW_cd", "Sector_agg",	"Commodity_NeW_nm", "Value_TJ"]]


In [ ]:
## Browse df_enr_NeW content
print(df_enr_NeW.columns)
for content in df_enr_NeW.columns:
    print(content, ": ", df_enr_NeW[content].unique())

In [ ]:
## To calculate "Net Conversion Efficiency NCE factors for thermal input"
# Calculation of electricity share (as transformation inputs for PG & HEAT together)
# Sub dataframe including PRODPELEC & PRODPHEAT
lst_inpu_pght_toexcl =  ['HYD', 'PV', 'WD', 'NUC', 'OTH', 'TID', 'ELEC', 'HP']
df_pg = df_enr_NeW.loc[(df_enr_NeW["Variable_NeW_cd"] == "PRODPELEC") & (~(df_enr_NeW["Commodity_NeW_nm"]).isin(lst_inpu_pght_toexcl))].copy()
df_ht = df_enr_NeW.loc[(df_enr_NeW["Variable_NeW_cd"] == "PRODPHEAT") & (~(df_enr_NeW["Commodity_NeW_nm"]).isin(lst_inpu_pght_toexcl))].copy()

df_pg.rename(columns={'Value_TJ': 'Value_TJ_ELEC'}, inplace= True)
df_ht.rename(columns={'Value_TJ': 'Value_TJ_HEAT'}, inplace= True)

# Merge both dataframes to have two value columns: Value_TJ_ELEC & Value_TJ_HEAT
df_proppght = pd.merge(
    df_pg[["Region_un_nm", "Region_NeW_cd", "Commodity_NeW_nm", "Value_TJ_ELEC"]],
    df_ht[["Region_un_nm", "Region_NeW_cd", "Commodity_NeW_nm", "Value_TJ_HEAT"]],
    on=['Region_un_nm', 'Region_NeW_cd', "Commodity_NeW_nm"],
    how='outer')

# Replace NaN values in column "Value_TJ_HEAT" by zero
df_proppght.loc[df_proppght["Value_TJ_HEAT"].isna(), "Value_TJ_HEAT"] = 0

# Calculation of ELEC share for split in DINTENP
df_proppght["Share_ELEC"] = df_proppght["Value_TJ_ELEC"].fillna(0)/(df_proppght["Value_TJ_ELEC"].fillna(0)+df_proppght["Value_TJ_HEAT"].fillna(0))

# Dataframe collecting DINTENP for ELEC & PG
df_dintenpbytech = df_enr_NeW.loc[(df_enr_NeW["Variable_NeW_cd"] == "DINTENP") & ((df_enr_NeW["Sector_agg"] == "PG") | (df_enr_NeW["Sector_agg"] == "HT"))].copy()
df_dintenpbytech.drop(columns=["Variable_un_nm", "Variable_NeW_cd"], inplace = True)


df_dintenpbytech.rename(columns={"Value_TJ": "Value_TJ_DINTENP"}, inplace = True)
df_dintenpbytech_piv = df_dintenpbytech.pivot_table(index = ["Region_un_nm", "Region_NeW_cd", "Commodity_NeW_nm"], columns="Sector_agg", values="Value_TJ_DINTENP").reset_index()
df_dintenpbytech_piv.rename(columns={"PG": "Value_PG_DINTENP"}, inplace = True)
df_dintenpbytech_piv.rename(columns={"HT": "Value_HT_DINTENP"}, inplace = True)



# Aggregattion of commodities MW & IW into WST
eb_tmp_wst = df_dintenpbytech_piv[df_dintenpbytech_piv["Commodity_NeW_nm"].isin(["IW", "MW"])].copy()
eb_tmp_no_wst = df_dintenpbytech_piv[~df_dintenpbytech_piv["Commodity_NeW_nm"].isin(["IW", "MW"])].copy()
eb_tmp_grp = (eb_tmp_wst.groupby(["Region_un_nm", "Region_NeW_cd"], as_index=False)[["Value_PG_DINTENP", "Value_HT_DINTENP"]].sum())
eb_tmp_grp["Commodity_NeW_nm"] = "WST"
eb_tmp_grp = eb_tmp_grp[["Region_un_nm", "Region_NeW_cd", "Commodity_NeW_nm", "Value_PG_DINTENP", "Value_HT_DINTENP"]]


df_dintenpbytech_agg = pd.concat([eb_tmp_no_wst, eb_tmp_grp], ignore_index=True)   

# Merge both dataframes (PRODP & DINTENP)
df_calc_nce_factors = pd.merge(
    df_proppght,
    df_dintenpbytech_agg,
    on=["Region_un_nm", 'Region_NeW_cd', "Commodity_NeW_nm"],
    how= "outer")

df_calc_nce_factors[["Region_un_nm", "Region_NeW_cd", "Commodity_NeW_nm", "Value_PG_DINTENP", "Value_TJ_ELEC", "Value_HT_DINTENP", "Value_TJ_HEAT", "Share_ELEC"]]

# Calculation of "Net Conversion Efficiency NCE factors for thermal inputs" by region
# df_calc_nce_factors["ELEC NCE"] = df_calc_nce_factors["Value_TJ_ELEC"]/(df_calc_nce_factors["Value_PG_DINTENP"]*df_calc_nce_factors["Share_ELEC"])
# df_calc_nce_factors["HEAT NCE"] = df_calc_nce_factors["Value_TJ_HEAT"]/(df_calc_nce_factors["Value_HT_DINTENP"]*(1-df_calc_nce_factors["Share_ELEC"]))
df_calc_nce_factors["ELEC NCE"] = df_calc_nce_factors["Value_TJ_ELEC"]/(df_calc_nce_factors["Value_PG_DINTENP"])
df_calc_nce_factors["HEAT NCE"] = df_calc_nce_factors["Value_TJ_HEAT"]/(df_calc_nce_factors["Value_HT_DINTENP"])

# Remplace NaN values in HEAT NCE by value in ELEC NCE
df_calc_nce_factors.loc[df_calc_nce_factors["HEAT NCE"].isna(), "HEAT NCE"] = df_calc_nce_factors["ELEC NCE"]

df_nce_factors = df_calc_nce_factors.loc[~((df_calc_nce_factors["HEAT NCE"] >1) | (df_calc_nce_factors["ELEC NCE"] >1))].copy()

df_nce_factors.drop(columns=["ELEC NCE", "HEAT NCE", "Share_ELEC"], inplace=True)

df_nce_factors = df_nce_factors.groupby(by=["Region_NeW_cd", "Commodity_NeW_nm"], as_index = False)[["Value_TJ_ELEC", "Value_TJ_HEAT", "Value_HT_DINTENP", "Value_PG_DINTENP"]].sum()

df_nce_factors["ELEC NCE"] = df_nce_factors["Value_TJ_ELEC"]/(df_nce_factors["Value_PG_DINTENP"])
df_nce_factors["HEAT NCE"] = df_nce_factors["Value_TJ_HEAT"]/(df_nce_factors["Value_HT_DINTENP"])

# Remplace NaN values in HEAT NCE by value in ELEC NCE
df_nce_factors.loc[df_nce_factors["HEAT NCE"].isna(), "HEAT NCE"] = df_nce_factors["ELEC NCE"]

# Calculate mean value by commodity
df_ncebycmd = df_nce_factors[["Commodity_NeW_nm","ELEC NCE","HEAT NCE"]].groupby(by=["Commodity_NeW_nm"], as_index=False).mean()

# Remplace NaN values in ELEC NCE by mean values
df_nce_tmp = df_nce_factors.merge(df_ncebycmd, on="Commodity_NeW_nm", how="left",
    suffixes=("", "_from_df_ncebycmd"))

for col in ["ELEC NCE", "HEAT NCE"]:
    df_nce_tmp[col] = df_nce_tmp[col].fillna(df_nce_tmp[f"{col}_from_df_ncebycmd"])

df_nce_factors = df_nce_tmp[df_nce_factors.columns].copy()

df_nce_factors.drop(columns=["Value_TJ_ELEC", "Value_TJ_HEAT", "Value_HT_DINTENP", "Value_PG_DINTENP"], inplace=True)

# Export Net Conversion Efficiency values in a csv file
path_exp = "E:/Work/DIAMOND/NeW/Data_Raw/Energy/"
nm_file = "Net_Efficiency_Conversion_UnData.csv"
path = path_exp + nm_file
df_nce_factors.to_csv(path, sep =";")


In [ ]:
## To calculate share to split Omnia regions
# Omnia regions to split 

lst_reg_ANZ = {"Australia": ["Australia", "New Zealand"]}
lst_reg_IDN ={"Indonesia": ["Indonesia", "Philippines", "Viet Nam"]}
lst_reg_ENW  = {"United Kingdom": ["Norway including Svalbard and Jan Mayen Islands", "Switzerland-Liechtenstein", "Iceland", "United Kingdom",
             "Gibraltar", "Saint Helena"]}
lst_reg_EUW  = {"Germany": ["Germany", "Netherlands", "Belgium", "Sweden", "Austria", "Finland", "Denmark", "Ireland", "Luxembourg", "Greenland", "Faeroe Islands", "Andorra"]}
lst_reg_EUM_1 = {"France incl. Monaco": ["France incl. Monaco", "Italy and San Marino", "Spain", "Greece", "Portugal", "Cyprus", "Malta"]}
lst_reg_EUM_2 = {"Italy and San Marino": ["France incl. Monaco", "Italy and San Marino", "Spain", "Greece", "Portugal", "Cyprus", "Malta"]}
lst_reg_EUM_3 = {"Spain": ["France incl. Monaco", "Italy and San Marino", "Spain", "Greece", "Portugal", "Cyprus", "Malta"]}

lst_reg_tosplit = [lst_reg_ANZ, lst_reg_IDN, lst_reg_ENW, lst_reg_EUW, lst_reg_EUM_1, lst_reg_EUM_2, lst_reg_EUM_3]

# Mapping sectors in UnData and OMNIA
dict_dfinenp_map_sect = {"IND_CHM": "ICH", "IND_CONS": 'ICM', "IND_I&S": 'IIS', "IND_NMMIN": 'INM', "IND_NFMET": 'INF',
                         "IND_FOOD": 'IOI', "IND_MACH": 'IOI', "IND_M&Q": 'IOI', "IND_PPP": 'IOI', "IND_TREQU": 'IOI', 
                         "IND_WWP": 'IOI', "IND_OTH": 'IOI',  'IND_TEXT': "IOI",
                         'TR_ROAD': 'ROA', 'TR_RAIL': 'RAI',  "TR_NAVDOM": 'NAD', "TR_AIRINT": 'AVB', "TR_AIRDOM": 'AVD', "TR_NAVINT": 'NAB', "TR_OTH": "OTH_TR", "TR_PIP": 'Pipeline',
                         'RES': "RES", "SERV": 'SRV', "AFOFI": 'AGR', "OTH": "OTH",
                         'OILGASM': "OTH OwnUses", 'REFOIL': "OTH OwnUses", 'LNG': "OTH OwnUses", 'PUMP': "PWR OwnUses", 'GASSEP': "OTH OwnUses", 'COALM': "OTH OwnUses",
                         'BQTPLT': "OTH OwnUses", "OWU_COALL": "OTH OwnUses", 'NUCF': "PWR OwnUses", 'PG':  "PWR OwnUses", 'HT': "PWR OwnUses", 'OWU_BG': "OTH OwnUses", 
                         "OWU_OTH": "OTH OwnUses", "COKOVENS": "OTH OwnUses", "BLASTF": "OTH OwnUses", "GASWRK": "OTH OwnUses", "CHARCP": "OTH OwnUses"}

dict_dintenp_map_sect = {"BLASTF": 'Blast furnace', "COKOVENS": 'Coke oven', "GTL": 'Liquefation', 'GASWRK': "Other", "TIN_COALL": 'Liquefation', "CHARCP": "Other", 'GASBLD': "Other",
                         'PETROCH': "Other", 'PG': "PG", 'HT': "HEAT", "PG": "PG", 'REFOIL': "Refineries", "BQTPLT": "Other", 'OTH': "Other"}

dict_prodp_map_sect = {'COMB': "Production", 'GAS': "Production", 'HYD': "Production", 'OIL': "Production", 'PV': "Production", 'SBM': "Production", 
                       'WD': "Production", 'MW': "Production", 'LBF': "Production", 'NUC': "Production", 'BG': "Production", 
                       'IW': "Production", 'OTH_URN': "Production", 'GEO': "Production", 'TID': "Production"}

dict_dnonenp_map_sect = {'OIL': "Non energy", 'GAS': "Non energy", 'COMB': "Non energy"}


dict_oth_map_sect = {"BG" : "BG", 'COMB' : "COMB", 'GAS': "GAS", 'HYD': "HYD", 'OIL': "OIL", 'PV': "PV", 'SBM': "SBM", 'WD': "WD", 
                    'MW': "MW", 'LBF': "LBF", 'NUC': "NUC", 'IW': "IW", 'GEO': "GEO", 'TID':" TID", 'OTH_URN' : "NUC",
                    "ELEC": "ELEC", 'HEAT' : "HEAT"}

df_enr_NeW_trav = df_enr_NeW.copy().reset_index(drop=True)

# Add OMNIA Sector
df_enr_NeW_trav.loc[df_enr_NeW_trav["Variable_NeW_cd"].str.startswith("DFINENP"), "Sector_OMNIA"] = df_enr_NeW_trav["Sector_agg"].map(dict_dfinenp_map_sect)
df_enr_NeW_trav.loc[df_enr_NeW_trav["Variable_NeW_cd"].str.startswith("DINTENP"), "Sector_OMNIA"] = df_enr_NeW_trav["Sector_agg"].map(dict_dintenp_map_sect)
df_enr_NeW_trav.loc[(df_enr_NeW_trav["Variable_NeW_cd"] == "PRODP"), "Sector_OMNIA"] = df_enr_NeW_trav["Sector_agg"].map(dict_prodp_map_sect)
df_enr_NeW_trav.loc[(df_enr_NeW_trav["Variable_NeW_cd"] == "DNONENP"), "Sector_OMNIA"] = df_enr_NeW_trav["Sector_agg"].map(dict_dnonenp_map_sect)

lst_oth_var = ["IMPP", "EXPP", "DSTOCKP"]
for var in lst_oth_var:
    mask = df_enr_NeW_trav["Variable_NeW_cd"].str.startswith(f"{var}")
    df_enr_NeW_trav.loc[mask, "Sector_OMNIA"] = df_enr_NeW_trav.loc[mask, "Sector_agg"].map(dict_oth_map_sect)

df_enr_NeW_trav = df_enr_NeW_trav.groupby(by=["Region_un_nm", "Region_NeW_cd", "Variable_un_nm", 
                                              "Variable_NeW_cd", "Sector_OMNIA", "Commodity_NeW_nm"], as_index=False)["Value_TJ"].sum()



# Calculate for each subset the share of the variable to split
df_eb_sh = pd.DataFrame()
share_cols = []
for lst in lst_reg_tosplit:
    for reg, agg_reg in lst.items():
        share_cols.append(f"Share_{reg}")
        # sub dataframe with data for region to split
        df_reg = df_enr_NeW_trav.loc[df_enr_NeW_trav["Region_un_nm"] == f"{reg}"].copy()
        df_reg.rename(columns={"Value_TJ": f"Value_TJ_Reg"}, inplace=True)
        # df_reg = df_reg.groupby(by=["Region_NeW_cd", "Commodity_NeW_nm", "Variable_un_nm", "Variable_NeW_cd", "Sector_agg"], as_index=False)["Value_TJ_Reg"].sum()
        # sub dataframe with data for Ominia aggregate regions
        df_agg_reg = df_enr_NeW_trav.loc[df_enr_NeW_trav["Region_un_nm"].isin(agg_reg)].copy()
        # Sum data by Ominia aggregate region
        df_agg = df_agg_reg.groupby(by=["Commodity_NeW_nm", "Variable_un_nm", 'Variable_NeW_cd', 'Sector_OMNIA'], as_index = False)["Value_TJ"].sum()
        df_agg.rename(columns={"Value_TJ": f"Value_TJ_RegAgg"}, inplace=True)
        # Merge both dataframes
        df_merged = pd.merge(df_reg[["Variable_un_nm", "Variable_NeW_cd", "Sector_OMNIA", "Commodity_NeW_nm", "Value_TJ_Reg"]],
                            df_agg[["Variable_un_nm", "Variable_NeW_cd", "Sector_OMNIA", "Commodity_NeW_nm", "Value_TJ_RegAgg"]], 
                        on = ["Variable_un_nm", "Variable_NeW_cd", "Sector_OMNIA", "Commodity_NeW_nm"], how = 'outer')
        df_merged = df_merged.groupby(by=["Variable_NeW_cd", "Sector_OMNIA", "Commodity_NeW_nm"], as_index=False)[["Value_TJ_Reg", "Value_TJ_RegAgg"]].sum()
        # Calculate the share of the region to split in Ominia aggregate region
        df_merged[f"Share_{reg}"] = df_merged["Value_TJ_Reg"]/df_merged["Value_TJ_RegAgg"]
        # Replace NaN by 0 for columns "Share_{reg}"
        df_merged.loc[df_merged[f"Share_{reg}"].isna(), f"Share_{reg}"] = 0
        if df_eb_sh.empty: # for the 1st loop 
            df_eb_sh = df_merged.copy()
        else: # merge each temporary dataframe into a final dataframe "df_eb_sh" excluding columns: "Value_TJ_Reg" and "Value_TJ_RegAgg"
            df_eb_sh = pd.merge(df_eb_sh[[col for col in df_eb_sh.columns if col not in ["Value_TJ_Reg", "Value_TJ_RegAgg"]]], 
                                df_merged[[col for col in df_merged.columns if col not in ["Value_TJ_Reg", "Value_TJ_RegAgg"]]],
                                on = ["Variable_NeW_cd",	"Sector_OMNIA", "Commodity_NeW_nm"], how = "outer")

for lst in lst_reg_tosplit: # replace NaN value by zero
    for reg in lst.keys():
        df_eb_sh.loc[df_eb_sh[f"Share_{reg}"].isna(), f"Share_{reg}"] = 0 


# Convert into an usable format for Omnia data
# share_cols = ['Share_AU', 'Share_ID', 'Share_KR', 'Share_UK', 'Share_DE', 'Share_FR', 'Share_IT', 'Share_ES']

df_eb_sh_mlt = df_eb_sh.melt(id_vars=['Variable_NeW_cd', 'Sector_OMNIA', 'Commodity_NeW_nm'],
                        value_vars=share_cols, var_name='Region', value_name='Share')

df_eb_sh_mlt['Region_splitted'] = df_eb_sh_mlt['Region'].str[6:]
df_eb_sh_mlt = df_eb_sh_mlt.drop(columns='Region')
df_eb_sh_mlt["Variable_NeW"] = df_eb_sh_mlt["Variable_NeW_cd"]+ df_eb_sh_mlt["Commodity_NeW_nm"]


df_omnia_reg_sh = df_eb_sh_mlt[["Region_splitted", "Variable_NeW", "Sector_OMNIA", "Share"]]

dict_map_reg_splitted = {'Australia': "AU", 'Indonesia': "ID", 'United Kingdom' : "UK",
       'Germany': "DE", 'France incl. Monaco': "FR", 'Italy and San Marino' :"IT", 'Spain': "ES"}

df_omnia_reg_sh.loc[:, 'Region_splitted'] = df_omnia_reg_sh['Region_splitted'].replace(dict_map_reg_splitted)

path_exp = "E:/Work/DIAMOND/NeW/Data_Raw/Energy/"
nm_file = "Shares_tosplit_Omnia_Region.csv"
path = path_exp + nm_file
df_omnia_reg_sh.to_csv(path, sep=";", index=False)